In [1]:
import pandas as pd
import re
from Utility import extract_ontology_info, fix_usage

Preparation of a dataset of drugs belonging to AIFA List of equivalent medicines.

In [2]:
eq_drugs = None 

with open(
        'DrugData/Lista_farmaci_equivalenti.csv', 
        'r', 
        encoding='latin1'
) as f:
    for line in f.readlines():
        l = re.sub(r'[^\x00-\x7F]+', '', line)
        l = re.sub(r'\s+', ' ', l)
        l = re.sub(r'\n', '', l)
        l = re.sub(r'\*', '', l)
        l = re.sub(r'\'', '_', l)
        l = re.sub(r'\+', 'and', l)
        l = re.sub(r'\(', '', l)
        l = re.sub(r'\)', '', l)
        
        if eq_drugs is None:
            l = l.split(';')
            eq_drugs = pd.DataFrame(columns=l)
        else:
            l = l.lower()
            l = l.split(';')
            eq_drugs.loc[len(eq_drugs)] = l

eq_drugs.head()

,Principio attivo,Confezione di riferimento,ATC,AIC,Farmaco,Confezione,Ditta,Prezzo riferimento SSN,Prezzo Pubblico 14 aprile 2025,Differenza,Nota,Codice gruppo equivalenza
0,acarbosio,40 unita_ 100 mg - uso orale,a10bf01,44155024,acarbosio doc generici,"""""""100 mg compresse"""" 40 compresse in blister ...",doc generici s.r.l.,"5,63","5,63","0,00",,h1a
1,acarbosio,40 unita_ 100 mg - uso orale,a10bf01,39716182,acarbosio tecnigen,"""""""100 mg compresse"""" 40 compresse in blister ...",tecnimede sociedade tecnico-medicinal s.a.,"5,63","5,63","0,00",,h1a
2,acarbosio,40 unita_ 100 mg - uso orale,a10bf01,38835144,acarphage,"""""""100 mg compresse"""" 40 compresse in blister ...",bruno farmaceutici s.p.a.,"5,63","5,63","0,00",,h1a
3,acarbosio,40 unita_ 100 mg - uso orale,a10bf01,26851016,glucobay,"""""""100 mg compresse"""" 40 compresse""",bayer s.p.a.,"5,63","8,04","2,41",,h1a
4,acarbosio,40 unita_ 50 mg - uso orale,a10bf01,47612027,acarbosio aurobindo,"""""""50 mg compresse"""" 40 compresse in blister p...",aurobindo pharma italia s.r.l.,"5,63","5,63","0,00",,h1b


In [3]:
eq_drugs[['Confezione','Uso']] = eq_drugs[
    'Confezione di riferimento'].str.split('uso', expand=True)

eq_drugs.drop(['Confezione', 'Confezione di riferimento'], inplace=True, axis=1)
eq_drugs.head()

,Principio attivo,ATC,AIC,Farmaco,Ditta,Prezzo riferimento SSN,Prezzo Pubblico 14 aprile 2025,Differenza,Nota,Codice gruppo equivalenza,Uso
0,acarbosio,a10bf01,44155024,acarbosio doc generici,doc generici s.r.l.,"5,63","5,63","0,00",,h1a,orale
1,acarbosio,a10bf01,39716182,acarbosio tecnigen,tecnimede sociedade tecnico-medicinal s.a.,"5,63","5,63","0,00",,h1a,orale
2,acarbosio,a10bf01,38835144,acarphage,bruno farmaceutici s.p.a.,"5,63","5,63","0,00",,h1a,orale
3,acarbosio,a10bf01,26851016,glucobay,bayer s.p.a.,"5,63","8,04","2,41",,h1a,orale
4,acarbosio,a10bf01,47612027,acarbosio aurobindo,aurobindo pharma italia s.r.l.,"5,63","5,63","0,00",,h1b,orale


In [4]:
eq_drugs['Uso'] = eq_drugs['Uso'].str.strip()
eq_drugs['Uso'].unique()

array(['orale', 'parenterale', 'parenterale e orale',
       'orale rilascio prolungato', 'rettale', 'respiratorio',
       'respiratorio aerosol extrafine', 'respiratorio polvere',
       'respiratorio soluzione pressurizzata', 'oftalmico soluzione',
       'oftalmico', None, 'dermatologico', 'orale rp',
       'dermatologico shampoo', 'dermatologico unguento',
       'orale preparazione solida', 'rale', 'respiratorio soluzione',
       'orale rilascio modificato', 'parenterale ev', '- orale',
       'orale o vaginale', 'oftalmico gel monodose',
       'oftalmico senza conservanti', 'oftalmico soluzione multidose',
       'oftalmico soluzione monodose'], dtype=object)

In [5]:
eq_drugs.drop(
    ['Prezzo Pubblico 14 aprile 2025', 'Prezzo riferimento SSN', 'Differenza', 'Nota', 'AIC'],
    axis=1,
    inplace=True
)
eq_drugs.head()

,Principio attivo,ATC,Farmaco,Ditta,Codice gruppo equivalenza,Uso
0,acarbosio,a10bf01,acarbosio doc generici,doc generici s.r.l.,h1a,orale
1,acarbosio,a10bf01,acarbosio tecnigen,tecnimede sociedade tecnico-medicinal s.a.,h1a,orale
2,acarbosio,a10bf01,acarphage,bruno farmaceutici s.p.a.,h1a,orale
3,acarbosio,a10bf01,glucobay,bayer s.p.a.,h1a,orale
4,acarbosio,a10bf01,acarbosio aurobindo,aurobindo pharma italia s.r.l.,h1b,orale


In [6]:
eq_drugs['Codice gruppo equivalenza '] = eq_drugs['Codice gruppo equivalenza '].str.strip()
eq_drugs = eq_drugs.groupby(
    ['Principio attivo', 'ATC', 'Farmaco','Uso']
).agg({
    'Codice gruppo equivalenza ': lambda x: list(set(x)),
    'Ditta': lambda x: list(set(x))
}).reset_index()


eq_drugs.rename(columns={
    'Codice gruppo equivalenza': 'Gruppi Equivalenza'
}, inplace=True)

eq_drugs.head()

,Principio attivo,ATC,Farmaco,Uso,Codice gruppo equivalenza,Ditta
0,acarbosio,a10bf01,acarbosio aurobindo,orale,[h1b],[aurobindo pharma italia s.r.l.]
1,acarbosio,a10bf01,acarbosio doc generici,orale,"[h1a, h1b]",[doc generici s.r.l.]
2,acarbosio,a10bf01,acarbosio tecnigen,orale,"[h1a, h1b]",[tecnimede sociedade tecnico-medicinal s.a.]
3,acarbosio,a10bf01,acarphage,orale,"[h1a, h1b]",[bruno farmaceutici s.p.a.]
4,acarbosio,a10bf01,glucobay,orale,"[h1a, h1b]","[farma 1000 s.r.l., new pharmashop s.r.l., gmm..."


In [7]:
info_drugs = extract_ontology_info('drugsOntology.gbs')['Drugs']
print(info_drugs)

{'opt_usage': ['oral', 'sublingual', 'intravenous', 'intramuscular', 'parenteral', 'parenteralandoral', 'rectal'], 'attributes': ['drug', 'activeIngredient', 'atc', 'company', 'usage', 'equivalenceGroupCode', 'notes']}


In [8]:
mapping_data = {
    'Principio attivo': 'activeIngredient',
    'Farmaco': 'drug',
    'ATC': 'atc',
    'Ditta': 'company',
    'Codice gruppo equivalenza ': 'equivalenceGroupCode',
    'Uso': 'usage'
}

eq_drugs = eq_drugs.rename(columns=mapping_data)
eq_drugs.head()

,activeIngredient,atc,drug,usage,equivalenceGroupCode,company
0,acarbosio,a10bf01,acarbosio aurobindo,orale,[h1b],[aurobindo pharma italia s.r.l.]
1,acarbosio,a10bf01,acarbosio doc generici,orale,"[h1a, h1b]",[doc generici s.r.l.]
2,acarbosio,a10bf01,acarbosio tecnigen,orale,"[h1a, h1b]",[tecnimede sociedade tecnico-medicinal s.a.]
3,acarbosio,a10bf01,acarphage,orale,"[h1a, h1b]",[bruno farmaceutici s.p.a.]
4,acarbosio,a10bf01,glucobay,orale,"[h1a, h1b]","[farma 1000 s.r.l., new pharmashop s.r.l., gmm..."


In [9]:
eq_drugs = fix_usage(eq_drugs, info_drugs['opt_usage'])
eq_drugs = eq_drugs[info_drugs['attributes']]
eq_drugs = eq_drugs.fillna('_')
eq_drugs['equivalenceGroupCode'] = eq_drugs['equivalenceGroupCode'].astype(str)
eq_drugs['company'] = eq_drugs['company'].astype(str)

eq_drugs.head()

  0%|          | 0/3649 [00:00<?, ?it/s]

HTTPSConnectionPool(host='translate.google.com', port=443): Max retries exceeded with url: /m?tl=en&sl=it&q=orale (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x10face530>, 'Connection to translate.google.com timed out. (connect timeout=None)'))


,drug,activeIngredient,atc,company,usage,equivalenceGroupCode,notes
0,acarbosio aurobindo,acarbosio,a10bf01,['aurobindo pharma italia s.r.l.'],oral,['h1b'],_
1,acarbosio doc generici,acarbosio,a10bf01,['doc generici s.r.l.'],oral,"['h1a', 'h1b']",_
2,acarbosio tecnigen,acarbosio,a10bf01,['tecnimede sociedade tecnico-medicinal s.a.'],oral,"['h1a', 'h1b']",_
3,acarphage,acarbosio,a10bf01,['bruno farmaceutici s.p.a.'],oral,"['h1a', 'h1b']",_
4,glucobay,acarbosio,a10bf01,"['farma 1000 s.r.l.', 'new pharmashop s.r.l.',...",oral,"['h1a', 'h1b']",_


In [10]:
eq_drugs

,drug,activeIngredient,atc,company,usage,equivalenceGroupCode,notes
0,acarbosio aurobindo,acarbosio,a10bf01,['aurobindo pharma italia s.r.l.'],oral,['h1b'],_
1,acarbosio doc generici,acarbosio,a10bf01,['doc generici s.r.l.'],oral,"['h1a', 'h1b']",_
2,acarbosio tecnigen,acarbosio,a10bf01,['tecnimede sociedade tecnico-medicinal s.a.'],oral,"['h1a', 'h1b']",_
3,acarphage,acarbosio,a10bf01,['bruno farmaceutici s.p.a.'],oral,"['h1a', 'h1b']",_
4,glucobay,acarbosio,a10bf01,"['farma 1000 s.r.l.', 'new pharmashop s.r.l.',...",oral,"['h1a', 'h1b']",_
...,...,...,...,...,...,...,...
3644,zomig rapimelt,zolmitriptan,n02cc03,['grunenthal italia s.r.l.'],oral,['fub'],_
3645,zonegran,zonisamide,n03ax15,['amdipharm ltd'],oral,"['tic', 'tia', 'tib']",_
3646,zonisamide doc generici,zonisamide,n03ax15,['doc generici s.r.l.'],oral,"['tic', 'tia', 'tib']",_
3647,zonisamide sandoz,zonisamide,n03ax15,['sandoz s.p.a.'],oral,['tia'],_


In [11]:
with open("prolog/drugs.pl", "w", encoding="utf-8") as f:
    for _, row in eq_drugs.iterrows():
        name: str = row.get("drug")
        name = name.replace(' ','_')
        name = name.replace('.','_')
        
        if name:
            f.write(f'drug({name}).\n')
    for _, row in eq_drugs.iterrows():
        name: str = row.get("drug")
        name = name.replace(' ','_')
        name = name.replace('.','_')
        
        ingredient = row.get("activeIngredient")
        
        atc = row.get("atc")
        company = row.get("company")
        company = company.replace('[','')
        company = company.replace('"','')
        company = company.replace(']','')
        company = company.replace('\'','')
    
        usage = row.get("usage")
        
        code: str = row.get("equivalenceGroupCode")
        
        code = code.replace('[','')
        code = code.replace(']','')
        code = code.replace(',','')
        code = code.replace('"','')
        code = code.replace('\'','')
        note = row.get("notes")

        f.write(
            f'drug_data({name}, "{ingredient}", {atc}, "{company}", {usage}, "{code}", "{note}").\n\n'
        )

In [12]:
eq_drugs.to_csv('DrugData/DrugDataProcessed.csv', index=False)